Open and reshape data

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os import listdir
import shutil 
from scipy.stats import norm
import pdb


 RENAME RECTANGLE GAME FILES WITH DIEGO'S IDs 

In [ ]:

Id_ah=pd.read_excel('/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/protocolo/lista_estudiantes_ah.xlsx');      
Id_dlh=pd.read_excel('/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/protocolo/new_ids_diego.xlsx');

List_ah=[];
for i in range(0,len(Id_ah)):
    if not np.isnan(Id_ah['ID'][i]):
        List_ah.append([int(Id_ah['ID'][i]),Id_ah['Nombre y apellido'][i],Id_ah['Nombre y apellido'][i].lower().replace(' ','')])
        
List_dlh=[];
for i in range(0,len(Id_dlh)):
    List_dlh.append([int(Id_dlh['ID'][i]),Id_dlh['Nombre y apellido'][i],Id_dlh['Nombre y apellido'][i].lower().replace(' ',''),Id_dlh['Grupo'][i][0],Id_dlh['Gén.'][i]]);

List_ah=np.array(List_ah);
List_dlh=np.array(List_dlh);


COPY AND RENAME FILES

In [ ]:
where='/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data/';
to='/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/';

c=0
for f in listdir(where):
    idx1=f.find('_');
    idx2=f[idx1+1:].find('_')+idx1+1;
    Id_T=f[0:idx1];
    Id_L=f[idx1+1:idx2]   
    
    Id_T_new=List_dlh[List_dlh[:,2]==List_ah[List_ah[:,0]==Id_T,2],0];    
    Id_L_new=List_dlh[List_dlh[:,2]==List_ah[List_ah[:,0]==Id_L,2],0];    
    
    new_filename=str(Id_T_new[0])+'_'+str(Id_L_new[0])+f[idx2:];
    
    if Id_T != Id_T_new:
        print(Id_T,Id_T_new)
        print(f)
        print(new_filename)
        print('------------------')
    shutil.copyfile(where+f,to+new_filename)
    
    
    

 FUNCTION TO READ CSV DATA

In [ ]:
def read_data(where,f,List_dlh):
    Pair={};
    idx1=f.find('_');
    idx2=f[idx1+1:].find('_')+idx1+1;
    Id_T=f[0:idx1];
    Id_L=f[idx1+1:idx2];                    
    Pair['Teachers_Id']=int(Id_T);
    Pair['Learners_Id']=int(Id_L);    
    level_T=List_dlh[List_dlh[:,0]==Id_T,3];
    level_L=List_dlh[List_dlh[:,0]==Id_L,3];
    if level_T!=level_L:
        print('Warning: level_T!=level_L')
    
    Pair['Level']=int(level_T[0]);
    
    df = pd.read_csv(where+f, delimiter=','); 
    columns=df.columns.tolist();        
    tuples = [tuple(x) for x in df.values];
    Pair['Total_time']=int(tuples[-1][columns.index('time_elapsed')])/60000; # minutes
    Pair['Trials']=[];
    n_trial=0;    
    for row in tuples:         
        if (row[1]=='teacher' or row[1]=='learner') and isinstance(row[columns.index('Learners_Guess')], str) :            
            n_trial+=1;               
            examples=row[columns.index('Examples')];
            canvas_size=row[columns.index('canvas_size')];
            print(canvas_size)
            inside=row[columns.index('Inside')];
            inside=inside.split(',');
            examples_list=[];                
            if not(pd.isnull(examples)):
                examples=examples.split(",");            
                for element in examples:                
                    if element[0]=='[':
                        element=element[1:];
                    if element[-1]==']':
                        element=element[0:-1];
                    examples_list.append(float(element));
                examples_list = list(zip(*[iter(examples_list)] * 2));
            Examples=[];                
            i=0;
            for element in examples_list:
                Examples.append({'x':element[0],'y':element[1],'inside':inside[i]})
                i+=1;
            teachers_box=row[columns.index('Teachers_Box')];
            teachers_box=teachers_box[1:-1];
            teachers_box=teachers_box.split(",");   
            aux=[];
            for element in teachers_box:
                element=int(element);
                aux.append(element);
            teachers_box=aux;
            learners_guess=row[columns.index('Learners_Guess')];
            learners_guess=learners_guess.split(",");
            learners_guess_list=[];
            for element in learners_guess:
                if element[0]=='[':
                    element=element[1:];
                if element[-1]==']':
                    element=element[0:-1];
                learners_guess_list.append(float(element));
            learners_guess_list = list(zip(*[iter(learners_guess_list)] * 4))  
            won=row[columns.index('won')];
            Pair['Trials'].append({'Teachers_box':teachers_box,'Examples':Examples,'n_examples':len(Examples),'Learners_guess':learners_guess_list,'won':won});
    return Pair


CONCATENATE SEPARATED FILES

In [ ]:
where='/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/';

def cat_data(filetype): # filetype= 'learner' or 'teacher'   
    files_list=[];
    for f in listdir(where):
        if f[-4:]=='.csv':
            idx1=f.find('_');
            idx2=f[idx1+1:].find('_')+idx1+1;
            Id_T=f[0:idx1];
            Id_L=f[idx1+1:idx2]   
            if filetype in f:
                files_list.append(f)

    files_list=np.sort(files_list);

    new_list = [] # group files with same id's
    for f in files_list:
        idx1=f.find('_');
        idx2=f[idx1+1:].find('_')+idx1+1;
        if new_list and new_list[-1][0][0:idx2] == f[0:idx2]:
            new_list[-1].append(f)
        else:
            new_list.append([f])

    Data=[];
    for files in new_list:
        if len(files)==1:
            P=read_data(where,files[0],List_dlh);
            P['Trials'].pop(); # Remove last trial
            Data.append(P);
        if len(files)>1:
            P0=read_data(where,files[0],List_dlh);
            P0['Trials'].pop(); # Remove last trial before the crash
            for f in files[1:]:
                P=read_data(where,f,List_dlh);
                P0['Trials'].extend(P['Trials']);
            P0['Trials'].pop(); # Remove last trial
            Data.append(P0);
    return Data;


In [ ]:
Data_L=cat_data('learner');
Data_T=cat_data('teacher');

In [ ]:
for i in range(0,len(Data_L)):
    if Data_L[i]['Teachers_Id']==Data_T[i]['Teachers_Id']:        
        T_L=Data_L[i]['Trials'];
        T_T=Data_T[i]['Trials'];  
        if len(T_L)==len(T_T):
            for t in range(0,len(T_L)):
                if len(T_L[t]['Learners_guess'])!=len(T_T[t]['Learners_guess']):
                    print('Different Learners guess in trial: ',t,'of Teacher Id:',Data_T[i]['Teachers_Id'],'Learner Id:',Data_T[i]['Learners_Id'],'Couple idx: ',i)        
                if len(T_L[t]['Examples'])!=len(T_T[t]['Examples']):
                    print('Different Examples in trial: ',t,'of Teacher Id:',Data_T[i]['Teachers_Id'],'Learner Id:',Data_T[i]['Learners_Id'],'Couple idx: ',i)        
        if len(T_L)!=len(T_T):
            print('Different number of trials in Teachers_id:', Data_L[i]['Teachers_Id'],'index:',i)
    else: 
        print('missalignment')

# Correct missing trials in one of the two subjects
#Data_T[31]['Trials'].append(Data_L[31]['Trials'][-1])       
#Data_T[56]['Trials'].insert(11,Data_L[56]['Trials'][11]);
        

In [3]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#save_obj(Data_L,'Data')

In [ ]:
T_L=Data_L[56]['Trials'];
T_T=Data_T[56]['Trials'];

#T_T.insert(11,T_L[11]);

for i in range(0,len(T_L)):
    print('TRIAL:',i)
    print('LEARNER')
    print(T_L[i])
    print('TEACHER')
    print(T_T[i])                
    print('--------------------')
    

 ADD GENDER INFO

In [37]:


Data=load_obj('/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/Data');
for d in Data:
    g=List_dlh[List_dlh[:,0]==str(d['Teachers_Id']),4];    
    d['Teachers_gender']=g[0];
    
    g=List_dlh[List_dlh[:,0]==str(d['Learners_Id']),4];    
    d['Learners_gender']=g[0];    
    
#save_obj(Data,'/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/Data')    




In [6]:
Data=load_obj('/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/Data');
Data[0]['Trials'][0]

{'Examples': [{'inside': 'true', 'x': 108.5, 'y': 333.0},
  {'inside': 'false', 'x': 107.5, 'y': 228.0},
  {'inside': 'false', 'x': 110.5, 'y': 420.0},
  {'inside': 'false', 'x': 154.5, 'y': 320.0},
  {'inside': 'true', 'x': 70.5, 'y': 326.0}],
 'Learners_guess': [(74.0, 144.0, 316.0, 465.0),
  (80.0, 146.0, 259.0, 408.0),
  (67.0, 125.0, 255.0, 389.0),
  (79.0, 135.0, 263.0, 398.0),
  (61.0, 135.0, 250.0, 399.0)],
 'Teachers_box': [67, 152, 229, 416],
 'n_examples': 5,
 'won': False}

ADD CONSISTENCY OF LEARNERS GUESS WITH AVAILABLE DATA SAMPLES

In [16]:
Data=load_obj('/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/Data');


def check_consistency(Examples,box):
# Examples is a list of Examples coordinates and category (inside=true means circle. inside=false means cross)
# box is the box-tuple (xleft, xright, ytop, ybottom) drawn by the learner
    consistent=True
    for e in Examples:
        if (e['x']<box[0] or e['x']>box[1] or e['y']<box[2] or e['y']>box[3]) and e['inside']=='true':
            consistent=False
        elif not(e['x']<box[0] or e['x']>box[1] or e['y']<box[2] or e['y']>box[3]) and e['inside']=='false':
            consistent=False
    return consistent

for d in Data:
    for trial in d['Trials']:
        
        Examples=trial['Examples']
        
        Aux=trial['Learners_guess']
        Learners_guess_dict=[];
        idx=0
        for guess in Aux:
            idx+=1            
            consistent=check_consistency(Examples[0:idx],guess)
            Learners_guess_dict.append({'box':guess,'consistency':consistent})
            
        trial['Learners_guess']=Learners_guess_dict
    
#save_obj(Data,'/home/ahaimovici/Documents/Teaching/Rectangle_Game/San_Andres/Data_new_Id/Data')


In [15]:
Data[1]['Trials'][1]

{'Examples': [{'inside': 'true', 'x': 370.5, 'y': 415.0},
  {'inside': 'true', 'x': 426.5, 'y': 415.0},
  {'inside': 'false', 'x': 430.5, 'y': 468.0},
  {'inside': 'true', 'x': 366.5, 'y': 519.0},
  {'inside': 'false', 'x': 428.5, 'y': 524.0}],
 'Learners_guess': [{'box': (192.0, 540.0, 303.0, 442.0), 'consistency': True},
  {'box': (327.0, 529.0, 389.0, 476.0), 'consistency': True},
  {'box': (328.0, 529.0, 384.0, 458.0), 'consistency': True},
  {'box': (315.0, 431.0, 393.0, 532.0), 'consistency': False},
  {'box': (317.0, 425.0, 392.0, 536.0), 'consistency': False}],
 'Teachers_box': [366, 427, 414, 523],
 'n_examples': 5,
 'won': False}